<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../../data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [4]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [5]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [6]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [7]:
from scipy.sparse import csr_matrix

In [9]:
csr_matrix?

In [25]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [15]:
sites_flatten.shape[0]

3363580

In [45]:
full_sites_sparse.shape

(336358, 48371)

In [46]:
idx_split

253561

In [20]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [21]:
X_train_sparse.shape, y_train.shape, X_test_sparse.shape

((253561, 48371), (253561,), (82797, 48371))

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [57]:
LogisticRegression?

C:\Users\odnoperov\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1235: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 24 s


In [77]:
logit.predict_proba(X_test_sparse[:15, :])

array([[9.97789947e-01, 2.21005275e-03],
       [9.99999995e-01, 4.81099531e-09],
       [9.99999981e-01, 1.87303401e-08],
       [9.99999976e-01, 2.35512673e-08],
       [9.99968697e-01, 3.13032412e-05],
       [9.99781530e-01, 2.18469709e-04],
       [9.99452063e-01, 5.47936856e-04],
       [9.99867729e-01, 1.32271157e-04],
       [9.99204802e-01, 7.95198159e-04],
       [8.96866874e-01, 1.03133126e-01],
       [9.99969635e-01, 3.03654255e-05],
       [9.99913567e-01, 8.64331997e-05],
       [9.99562416e-01, 4.37583841e-04],
       [6.42289977e-01, 3.57710023e-01],
       [9.99949803e-01, 5.01973838e-05]])

In [67]:
X_test_sparse.shape

(82797, 48371)

In [22]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C=C, random_state=seed, n_jobs=-1)
    
    logit.fit(X_train,y_train)
    
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, valid_pred)

** Посмотрите какой получился ROC AUC на отложенной выборке **

In [24]:
%%time
get_auc_lr_valid(X_train_sparse,y_train)

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 20.9 s


0.9195267091268964

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [ ]:
%%time
logit = LogisticRegression(random_state=17, n_jobs=-1)
logit.fit (X_train_sparse, y_train)

In [82]:
test_pred = logit.predict_proba(X_test_sparse)[:, 1]

In [89]:
test_pred[:10], test_df.index[:10]

(array([2.21005275e-03, 4.81099531e-09, 1.87303401e-08, 2.35512673e-08,
        3.13032412e-05, 2.18469709e-04, 5.47936856e-04, 1.32271157e-04,
        7.95198159e-04, 1.03133126e-01]),
 Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='session_id'))

In [99]:
pd.Series(test_pred, index=range(1, test_pred.shape[0] + 1), name='target').to_csv('benchmark1.csv', header=True, index_label='session_id')

In [96]:
pd.Series(test_pred, index=range(1, test_pred.shape[0] + 1), name='target').head()

1    2.210053e-03
2    4.810995e-09
3    1.873034e-08
4    2.355127e-08
5    3.130324e-05
Name: target, dtype: float64

In [ ]:
# Ваш код здесь

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [82]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [ ]:
# Ваш код здесь

In [17]:
%%time
logit_full = LogisticRegression(random_state=17, n_jobs=-1, C=1.0)
logit_full.fit(X_train_sparse, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1296: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 20.1 s


In [39]:
full_test_predict = logit_full.predict_proba(X_test_sparse)[:,1]
full_test_predict

array([2.21005275e-03, 4.81099531e-09, 1.87303401e-08, ...,
       8.62823113e-03, 4.31275409e-04, 2.02769160e-05])

In [40]:
write_to_submission_file(full_test_predict, 'alice_submit_1.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [ ]:
# Ваш код здесь

In [26]:
time = ['time%d' % i for i in range(1, 11) ]
train_df[time].head()

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,NaT,NaT,NaT,NaT,NaT,NaT
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22


In [28]:
new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

In [29]:
new_feat_train['year_month'] = train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'] = test_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'].head()

session_id
1    201410
2    201407
3    201412
4    201411
5    201405
Name: year_month, dtype: int64

In [35]:
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1,1))

new_feat_train['year_month_scaler'] = scaler.transform(new_feat_train['year_month'].values.reshape(-1,1))
new_feat_test['year_month_scaler'] = scaler.transform(new_feat_test['year_month'].values.reshape(-1,1))

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [36]:
new_feat_train.head()

,year_month,year_month_scaler
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [ ]:
# Ваш код здесь

In [57]:
hstack?

In [38]:
X_train_sparse.shape, new_feat_train['year_month_scaler'].values.reshape(-1,1).shape

((253561, 48371), (253561, 1))

In [39]:
%%time
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, new_feat_train['year_month_scaler'].values.reshape(-1,1)]))

Wall time: 398 ms


In [76]:
%%time
X_test_sparse_new = csr_matrix(hstack([X_test_sparse, new_feat_test['year_month_scaler'].values.reshape(-1,1)]))

Wall time: 145 ms


In [40]:
X_train_sparse.shape, X_train_sparse_new.shape

((253561, 48371), (253561, 48372))

In [41]:
X_train_sparse_new.shape, y_train.shape

((253561, 48372), (253561,))

In [42]:
%%time
get_auc_lr_valid(X_train_sparse_new, y_train)

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 21.1 s


0.9196939657202479

In [43]:
print('После добавления нового признака year_month оценка качества модели на отложенной выборке увеличилась на '), 
0.9196939657202479 - 0.9195267091268964

После добавления нового признака year_month оценка качества модели на отложенной выборке увеличилась на 


0.00016725659335148357

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [ ]:
# Ваш код здесь

In [44]:
new_feat_train['start_hour'] = train_df['time1'].apply(lambda st: st.hour)
new_feat_test['start_hour'] = test_df['time1'].apply(lambda st: st.hour)

In [45]:
scaler2 = StandardScaler()
scaler2.fit(new_feat_train['start_hour'].values.reshape(-1,1))

new_feat_train['start_hour_scaler'] = scaler2.transform(new_feat_train['start_hour'].values.reshape(-1,1))
new_feat_test['start_hour_scaler'] = scaler2.transform(new_feat_test['start_hour'].values.reshape(-1,1))

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [46]:
new_feat_train.head()

,year_month,year_month_scaler,start_hour,start_hour_scaler
session_id,,,,
21669,201301,-1.744405,8,-1.357366
54843,201301,-1.744405,8,-1.357366
77292,201301,-1.744405,8,-1.357366
114021,201301,-1.744405,8,-1.357366
146670,201301,-1.744405,8,-1.357366


In [47]:
new_feat_train['morning'] = train_df['time1'].apply(lambda st: 1 if st.hour <= 11 else 0)
new_feat_test['morning'] = test_df['time1'].apply(lambda st: 1 if st.hour <= 11 else 0)

In [48]:
new_feat_train['morning'].value_counts()

0    131636
1    121925
Name: morning, dtype: int64

In [49]:
new_feat_train.head()

,year_month,year_month_scaler,start_hour,start_hour_scaler,morning
session_id,,,,,
21669,201301,-1.744405,8,-1.357366,1
54843,201301,-1.744405,8,-1.357366,1
77292,201301,-1.744405,8,-1.357366,1
114021,201301,-1.744405,8,-1.357366,1
146670,201301,-1.744405,8,-1.357366,1


In [50]:
new_feat_train['start_month'] = train_df['time1'].apply(lambda st: st.month)
new_feat_test['start_month'] = test_df['time1'].apply(lambda st: st.month)

In [51]:
new_feat_train['start_month'].value_counts()

3     59587
2     59492
1     36232
4     35201
11    34493
12    16552
9      3464
5      3229
10     2788
6      2094
8       290
7       139
Name: start_month, dtype: int64

In [52]:
new_feat_train.head()

,year_month,year_month_scaler,start_hour,start_hour_scaler,morning,start_month
session_id,,,,,,
21669,201301,-1.744405,8,-1.357366,1,1
54843,201301,-1.744405,8,-1.357366,1,1
77292,201301,-1.744405,8,-1.357366,1,1
114021,201301,-1.744405,8,-1.357366,1,1
146670,201301,-1.744405,8,-1.357366,1,1


**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`

In [56]:
%%time
X_train_sparse_start_month_start_hour = csr_matrix(hstack([X_train_sparse_new, new_feat_train['start_hour'].values.reshape(-1,1)]))

Wall time: 213 ms


In [77]:
%%time
X_test_sparse_start_month_start_hour = csr_matrix(hstack([X_test_sparse_new, new_feat_test['start_hour'].values.reshape(-1,1)]))

Wall time: 71 ms


In [54]:
X_train_sparse_new.shape, X_train_sparse_start_month_start_hour.shape

((253561, 48372), (253561, 48373))

In [55]:
%%time
auc_train_start_month_start_hour = get_auc_lr_valid(X_train_sparse_start_month_start_hour, y_train)
print('Точность увеличилась на: ', auc_train_start_month_start_hour - 0.9196939657202479)

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Точность увеличилась на:  0.037590919355754204
Wall time: 19.4 s


**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `morning`

In [58]:
%%time
X_train_sparse_start_month_morning = csr_matrix(hstack([X_train_sparse_new, new_feat_train['morning'].values.reshape(-1,1)]))

Wall time: 200 ms


In [59]:
X_train_sparse_new.shape, X_train_sparse_start_month_morning.shape

((253561, 48372), (253561, 48373))

In [62]:
%%time
auc_train_start_month_morning = get_auc_lr_valid(X_train_sparse_start_month_morning, y_train)
print('Точность увеличилась на: ', auc_train_start_month_morning - 0.9196939657202479)

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Точность увеличилась на:  0.028973430958048274
Wall time: 20.3 s


**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month`, `start_hour` и `morning`

In [60]:
%%time
X_train_sparse_start_month_hour_morning = csr_matrix(hstack([X_train_sparse_start_month_start_hour, new_feat_train['morning'].values.reshape(-1,1)]))

Wall time: 229 ms


In [78]:
%%time
X_test_sparse_start_month_hour_morning = csr_matrix(hstack([X_test_sparse_start_month_start_hour, new_feat_test['morning'].values.reshape(-1,1)]))

Wall time: 86 ms


In [61]:
X_train_sparse_new.shape, X_train_sparse_start_month_hour_morning.shape

((253561, 48372), (253561, 48374))

In [63]:
%%time
auc_train_start_month_hour_morning = get_auc_lr_valid(X_train_sparse_start_month_hour_morning, y_train)
print('Точность увеличилась на: ', auc_train_start_month_hour_morning - 0.9196939657202479)

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Точность увеличилась на:  0.03881696452831829
Wall time: 25.2 s


### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [65]:
%%time
base_score = get_auc_lr_valid(X_train_sparse_start_month_hour_morning, y_train)
base_score

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 25.1 s


In [ ]:
# Ваш код здесь

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [68]:
#видоизменим исходную функцию в новую чтобы у нас была возможность передавать в неё в качестве аргумента коэфицент регуляризации
def get_auc_lr_valid_c(X, y, C, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C=C, random_state=seed, n_jobs=-1)
    
    logit.fit(X_train,y_train)
    
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, valid_pred)

In [67]:
np.logspace(-3,1,10)

array([1.00000000e-03, 2.78255940e-03, 7.74263683e-03, 2.15443469e-02,
       5.99484250e-02, 1.66810054e-01, 4.64158883e-01, 1.29154967e+00,
       3.59381366e+00, 1.00000000e+01])

In [69]:
%%time
max = 0
max_c = 0
for i in np.logspace(-3,1,10):
    mb = get_auc_lr_valid_c(X_train_sparse_start_month_hour_morning, y_train, i)
    if mb > max:
        max = mb
        max_c = i

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\U

Wall time: 3min 53s


In [70]:
max, max_c

(0.9588957109854371, 0.46415888336127775)

In [ ]:
# Ваш код здесь

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [75]:
X_train_sparse_start_month_hour_morning.shape, X_test_sparse.shape, test_df.shape

((253561, 48374), (82797, 48371), (82797, 20))

In [80]:
%%time
logit_3 = LogisticRegression(random_state=17, n_jobs=-1, C=max_c)
logit_3.fit(X_train_sparse_start_month_hour_morning, y_train)
test_predict_3_new_fet = logit_3.predict_proba(X_test_sparse_start_month_hour_morning)[:,1]
test_predict_3_new_fet

C:\Users\odnoperov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 31.4 s


In [83]:
write_to_submission_file(test_predict_3_new_fet, 'alice_submit_2.csv')

In [ ]:
# Ваш код здесь